In [22]:
from ipyleaflet import *
from ipyleaflet import AwesomeIcon, Marker, Map
import s3fs
import pyarrow as pa
import pyarrow.dataset as ds

import sys
import os

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd
import polars as pl
import altair as alt
import matplotlib.pyplot as plt

import geopandas as gpd

import datetime

from pyspark.sql.functions import *

from shapely.geometry import shape, Point
from ipywidgets import Layout

In [8]:
# load GeoJSON file containing sectors
with open('manhattan.geojson') as f:
    js = json.load(f)

In [19]:
def checkIfIn(cords:list):
    point=Point(cords[1],cords[0])
    # check each polygon to see if it contains the point
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            return True
    return False

[40.69625781921317, -74.02656555175783]

[40.69938133866613, -73.91361236572267]

[40.88834126500965, -73.89953613281251]

[40.881073537911995, -74.02210235595705]


In [27]:
m = Map(center=(40.7128, -74.0060), zoom=15,layout=Layout(height='1000px'))
#m.fit_bounds([[40.69625781921317, -74.02656555175783],[40.88834126500965, -73.89953613281251]])


click_pos=[]
markers=[]

icon_start = AwesomeIcon(
    name='play',
    marker_color='green',
    icon_color='white',
    spin=False
)

icon_stop = AwesomeIcon(
    name='ban',
    marker_color='red',
    icon_color='black',
    spin=False
)

def markers_handler(**kwargs):
    
    coords=kwargs.get('coordinates')
    print(coords)
    if(checkIfIn(coords)):

        click_pos.append(coords)

        
        if len(click_pos)==3:
            m.remove_layer(markers[0])
            m.remove_layer(markers[1])

            click_pos.pop(0)
            markers.pop(0)

            marker_start=Marker(location=click_pos[0],icon=icon_start)
            marker_stop=Marker(location=click_pos[1],icon=icon_stop)
            
            m.add(marker_start)
            m.add(marker_stop)

            markers[0]=marker_start
            markers.append(marker_stop)

        
        elif len(click_pos)==2:
            marker_stop=Marker(location=click_pos[1],icon=icon_stop)
            m.add(marker_stop)
            markers.append(marker_stop)

        else:
            marker_single=Marker(location=click_pos[0],icon=icon_start)
            m.add(marker_single)
            markers.append(marker_single)

    else:
        print("c'est pas dedans chef")

def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        markers_handler(**kwargs)

m.on_interaction(handle_click)

m

Map(center=[40.7128, -74.006], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

[40.74049401829621, -74.02965545654298]
c'est pas dedans chef
[40.739583569403294, -73.99978637695314]
[40.75401921961654, -73.98450851440431]
[40.73438076524639, -74.04270172119142]
c'est pas dedans chef
[40.746736760718505, -74.03514862060548]
c'est pas dedans chef
